# Classically-Boosted Variational Quantum Eigensolver

One of the most important applications of quantum computers is expected to be the computation of ground-state energies of complicated molecules and materials. Even though there are already some solid proposals on how to tackle these problems when fault-tolerant quantum computation comes into play, we currently live in the [`NISQ`](https://en.wikipedia.org/wiki/Noisy_intermediate-scale_quantum_era) era, meaning that we can only access very noisy and limited devices. That is why a large part of the research on quantum algorithms is focusing on what can be done with few resources. In particular, most proposals rely on variational quantum algorithms (VQA), which are optimized classically and adapt to the limitations of the quantum devices. For the specific problem of computing ground-state energies, this reduces to the very well-known Variational Quantum Eigensolver (VQE) algorithm.

[add picture-drawing VQE vs FT]

Although VQE is intended to run on NISQ devices, it is nonetheless sensitive to noise. This is in particular problematic when applying a large number of gates. As a consequence, several modifications to the original VQE algorithm have been proposed. These variants are usually intended to improve the algorithm's performance on NISQ-era devices. 

Here we will go through one of these proposals step-by-step: the Classically-Boosted Variational Quantum Eigensolver (CB-VQE), which was recently proposed in this [`paper`](https://arxiv.org/abs/2106.04755). Implementing CB-VQE allows to reduce the number of measurements required to obtain the ground-state energy with a certain precision. This is done by making use of classical states which already contain some information about the ground-state of the problem. 

In this demo, we will restrict ourselves to the $H_2$ molecule for the sake of simplicity. First, we will give a short introduction on how to perform standard VQE for the molecule of interest. For more details, we recommend the following [`demo`](https://pennylane.ai/qml/demos/tutorial_vqe.html) to learn how to implement VQE for molecules step-by-step. Then, we will implement the CB-VQE algorithm for the specific case in which we rely only on one classical state, that being the Hartree-Fock state. Finally, we will discuss the number of measurements needed to obtain a certain error-threshold by comparing the two methods. 

[add picture-drawing]

Let's get started!

## Prequisites: Standard VQE

If you are not already familiar with the VQE family of algorithms and wish to see how one can apply it the $H_2$ molecule, feel free to work through [`VQE overview demo`](https://pennylane.ai/qml/demos/tutorial_vqe.html) before reading this Section. Here, we will only briefly review the main idea behind standard VQE and highlight the important concepts in connection with CB-VQE.

The main goal of VQE is to find the ground energy of the Schrödinger equation 

$$H \vert \phi \rangle = E  \vert \phi \rangle$$ 

This corresponds to the problem of diagonalizing the Hamiltonian and finding the smallest eigenvalue. Alternatively, one can formulate the problem using the variational principle, in which we are interested in minimizing the energy 

$$E = \langle \phi \vert H \vert \phi \rangle$$ 

In VQE, we prepare a statevector $\vert \phi \rangle$ by applying a parameterized ansatz $A(\Theta)$ to an inital state $\vert 0^N \rangle$. Then, the parameters $\Theta$ are optimized to minimize a cost function which in this case is the energy

$$ E(\Theta) = \langle 0^{N} \vert A(\Theta)^{\dagger} H A(\Theta) \vert 0^{N} \rangle $$

This is done using a classical optimization method, which is typically gradient descent.

To implement our example of VQE, we first define the molecular Hamiltonian for the $H_2$ molecule in the minimal *STO-3G basis*  using PennyLane

In [ ]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np

symbols = ["H", "H"]
coordinates = np.array([0.0, 0.0, -0.6614, 0.0, 0.0, 0.6614])
basis_set = "sto-3g"
electrons = 2

H, qubits = qchem.molecular_hamiltonian(
    symbols,
    coordinates,
    basis=basis_set,
)

We then initialize the *Hartree-Fock* state $\vert \phi_{HF}\rangle=\vert 1100 \rangle$

In [ ]:
hf = qml.qchem.hf_state(electrons, qubits)

and implement the ansatz $A(\Theta)$. In this case, we use the [`AllSinglesDoubles`](https://pennylane.readthedocs.io/en/stable/code/api/pennylane.AllSinglesDoubles.html?highlight=allsinglesdoubles#pennylane.AllSinglesDoubles) class, which enables us to apply all possible combinations of single and double excitations obeying the Pauli principle to the *Hartree-Fock* state. Single and double excitation gates $G^{(1, 2)}(\Theta)$ are conveniently implemented in PennyLane with [`SingleExcitation`](https://pennylane.readthedocs.io/en/latest/code/api/pennylane.SingleExcitation.html) and [`DoubleExcitation`](https://pennylane.readthedocs.io/en/latest/code/api/pennylane.DoubleExcitation.html) classes. 

In [ ]:
singles, doubles = qml.qchem.excitations(electrons=electrons, orbitals=qubits)
num_theta = len(singles) + len(doubles)

def circuit_VQE(theta, wires):
    qml.AllSinglesDoubles(
        weights = theta,
        wires = wires,
        hf_state = hf,
        singles = singles,
        doubles = doubles)

Once this is defined, we can run the VQE algorithm. 
We first need to define a circuit for the cost function. For our purposes of studying the performance of VQE with the number of measurements, we will take a finite number of shots `num_shots = 100`

In [ ]:
num_shots = 100
dev = qml.device('default.qubit', wires=qubits, shots=int(num_shots))
@qml.qnode(dev)
def cost_fn(theta):
    circuit_VQE(theta,range(qubits))
    return qml.expval(H)

we then the classical optimization parameters `stepsize` and `max_iteration` 

In [ ]:
stepsize = 0.4
max_iterations = 10
opt = qml.GradientDescentOptimizer(stepsize=stepsize)
theta = np.zeros(num_theta, requires_grad=True)

and we finally run the algorithm

In [ ]:
for n in range(max_iterations):

    theta, prev_energy = opt.step_and_cost(cost_fn, theta)
    samples = cost_fn(theta)
           
energy_VQE = cost_fn(theta)
theta_opt = theta

print('VQE for num. of shots %.0f \nEnergy %.2f' %(num_shots, energy_VQE))

Note that as an output we obtain the VQE approximation to the ground state energy and a set of optimized parameters $\Theta$ that define the grounstate through the ansatz $A(\Theta)$. We will need to save these two quantities, as they are necessary to implement CB-VQE in the following steps.

## Classically-Boosted VQE

Now we are ready to present the classically-boosted version of VQE. 

The key of this new method relies on the notion of [`generalized eigenvalue problem`](https://en.wikipedia.org/wiki/Generalized_eigenvalue_problem). The main idea is to restrict the problem of finding the ground state to an eigenvalue problem in a subspace $\mathcal{H}^{\prime}$ of the complete Hilbert space $\mathcal{H}$. If this subspace is spanned by a combination of both classical and quantum states, we can run parts of our algorithm on classical hardware and thus reduce the number of measurements needed to reach a certain precision threshold. For a subspace spanned by the states $\{\vert \phi_\alpha \rangle\}_{\alpha\in \mathcal{H}^{\prime}}$, the generalized eigenvalue problem is expressed as 

$$\bar{H} \vec{v}=  \lambda \bar{S} \vec{v},$$

where $\bar{H}$ is the Hamiltonian $H$ projected into the subspace of interest, i.e. with the entries

$$\bar{H}_{\alpha, \beta} = \langle \phi_\alpha \vert H \vert \phi_\beta \rangle \quad \forall \alpha, \beta \in \mathcal{H}^{\prime} ,$$

and the matrix $\bar{S}$ contains the overlaps between the basis states. For a complete orthonormal basis, the overlap matrix $\hat{S}$ would simply be the identity matrix. However, we need to take a more general approach which works for a subspace spanned by potentially non-orthogonal states. We can retrieve the representation of $S$ in terms of $\{\vert \phi_\alpha \rangle\}_\alpha$ by calculating 

$$\bar{S}_{\alpha, \beta} = \langle \phi_\alpha \vert \phi_\beta \rangle \quad \forall \alpha, \beta \in \mathcal{H}^{\prime}.$$

Note that $\vec{v}$ and $\lambda$ are the eigenvectors and eigenvalues respectively. In particular, our goal is to find the lowest eigenvalue $\lambda_0$. 

Equipped with the useful mathematical description of generalized eigenvalue problems, we can now choose our subspace such that some of the states $\phi_{\alpha} \in \mathcal{H}^{\prime}$ are classically tractable. 

We will consider the simplest case in which the subspace is spanned only by one classical state $\vert \phi_{HF} \rangle$ and one quantum state $\vert \phi_{q} \rangle$. More precisely, we define the classical state to be a single [`Slater determinant`](https://en.wikipedia.org/wiki/Slater_determinant), which directly hints towards using the *Hartree-Fock* state for several reasons. First of all, it is well-known that the Hartree-Fock state is a good candidate to approximate the ground state in the mean-field limit. Secondly, we already compute it when we build the molecular Hamiltonian for the standard VQE!

To summarize, our goal is to build the hamiltonian $\bar{H}$ and the overlap matrix $\bar{S}$, which act on the subspace $\mathcal{H}^{\prime} \subseteq \mathcal{H}$ spanned by $\{\vert \phi_{HF} \rangle, \vert \phi_q \rangle\}$. These will be two-dimensional matrices, and in the following Sections we will show how to compute all their entries step by step.

As done previously, we start by importing *PennyLane*, *Qchem* and differentiable *NumPy* followed by defining the molecular hamiltonian in the Hartree-Fock basis for $H_2$.

In [ ]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np

# Define the molecular Hamiltonian
symbols = ["H", "H"]
coordinates = np.array([0.0, 0.0, -0.6614, 0.0, 0.0, 0.6614])
basis_set = "sto-3g"

H, qubits = qchem.molecular_hamiltonian(
    symbols,
    coordinates,
    basis=basis_set,
)

### Computing Classical Quantities

We first set out to calculate the purely classical part of the Hamiltonian $H$. Since we only have one classical state this will already correspond to a scalar energy value. The terms can be expressed as 

$$H_{11} = \langle \phi_{HF} \vert H \vert \phi_{HF} \rangle \quad \text{and} \quad S_{11} = \langle \phi_{HF} \vert \phi_{HF} \rangle$$

which is tractable using classical methods. This energy corresponds to the Hatree-Fock energy due to our convenient choice of the classical state. Note that the computation of the classical compononent of the overlap matrix $S_{11} = \langle \phi_{HF} \vert \phi_{HF} \rangle = 1$ is trivial. 

Using PennyLane, we can access the Hartree-Fock energy by looking at the fermionic Hamiltonian, which is the Hamiltonian in the basis of Slater determinants. The basis is organized in lexicographic order, meaning that if we want the entry corresponding to the Hartree Fock determinant $\vert 1100 \rangle$, we will have to take the entry $H_{i,i}$, where $1100$ is the binary representation of the index $i$.

In [ ]:
hf_state = qml.qchem.hf_state(electrons, qubits)
fermionic_Hamiltonian = qml.utils.sparse_hamiltonian(Hamiltonian).toarray()

binary_string = ''.join([str(i) for i in hf_state])
idx0 = int(binary_string, 2)
H11 = fermionic_Hamiltonian[idx0][idx0]
S11 = 1

### Computing Quatum Quantities

We now move on to the purely quantum part of the Hamiltonian, i.e. the entry 

$$H_{22} = \langle \phi_{q} \vert H \vert \phi_{q} \rangle,$$

where $\vert \phi_q \rangle$ is the quantum state.  This state is just the output of the standard VQE with a given ansatz, following the steps in the first Section. Therefore the entry $H_{22}$ just corresponds to the final energy of the VQE. In particular, note that the quantum state can be written as $\vert \phi_{q} \rangle = A(\theta^*) \vert \phi_{HF} \rangle$ where $A(\theta^*)$ is the ansatz of the VQE with the optimised parameters $\theta^*$. Once again, we have $S_{22}=\langle \phi_{q} \vert \phi_{q} \rangle = 1$ for the overlap matrix. 

In [ ]:
H22 = energy_VQE
S22 = 1

### Computing Mixed Quantities

The final part of the algorithm is computing the cross-terms between the classical and quantum state

$$H_{12} = \langle \phi_{HF} \vert H \vert \phi_{q} \rangle = H_{21}^{\dagger}.$$

This part of the algorithm is slightly more complicated than the previous steps, since we still want make use of the classical component of the problem in order to minimize the number of required shots. 

Keep in mind that most algorithms usually perform computations either on fully classically or quantum tractable Hilbert spaces. CB-VQE takes advantage of the classical part of the problem while still calculating a classically-intractable quantity by using the so-called [`Hadamard test`](https://en.wikipedia.org/wiki/Hadamard_test_(quantum_computation)) to construct $H_{12}$. The Hadamard test is a prime example of an indirect measurement, which allows us to measure properties of a state without (completely) destroying it [TODO:Cite]. In the case of the Hadamard test we can calculate the real part of $\langle \phi \rvert U \rvert \phi \rangle$ for some unitary $U$. We can also retrieve the imaginary part if we use a different initial state.

[TODO:INSERT IMAGE OF HADAMARD TEST CIRCUIT]

The Hadamard test onsists of the state to be projected on - which we labelled as $\rvert \phi \rangle$ - and an ancillary system prepared in the $\rvert 0 \rangle$ state. In the algorithm, a controlled version of the unitary $U$ is used. The ancilla qubit acts as the control. Two hadamard gates - one before and one after the control operation - are also applied to the ancillary system. 
In order to calculate $\langle \phi_{HF} \vert \phi_{q} \rangle$, we can choose $U$ such that $U\vert \phi_{q} \rangle = \vert \phi_{HF} \rangle$. 
To be able to calculate $H_{12}$, we must first decompose the hermitian operator into unitaries. Assuming we can write our Hamiltonian as $H = \sum_{l} h_{l}$, where $h_{l}$ is unitary, we simply apply the Hadamard test to $h_{l}$. 

In [ ]:
num_shots = 100
wires = range(qubits + 1)
dev = qml.device("default.qubit", wires=wires, shots=num_shots)

@qml.qnode(dev)
def hadamard_test(Vq, Vcl, W, component='real'):

    if component == 'imag':
        qml.RX(math.pi/2, wires=wires[1:])

    qml.Hadamard(wires=[0])
    qml.QubitUnitary(W, wires=wires[1:])
    qml.ControlledQubitUnitary(Vq.conjugate().T @ Vcl, control_wires=[0], wires=wires[1:])
    qml.Hadamard(wires=[0])

    return qml.probs(wires=[0])

Basis states!!

In [ ]:
def circuit_product_state(state):
    qml.BasisState(state, range(qubits))

Vcl = np.identity(2**qubits)
Vq = qml.matrix(circuit_VQE)(theta_opt, range(qubits))  

H12 = 0
relevant_basis_states = np.array([[1,1,0,0], [0,1,1,0], [1,0,0,1], [0,0,1,1]], requires_grad=True)
for j, basis_state in enumerate(relevant_basis_states):
    W = qml.matrix(circuit_product_state)(basis_state)
    probs = hadamard_test(Vq, Vcl, W)
    y = 2*abs(probs[0])-1
    binary_string = ''.join([str(coeff) for coeff in basis_state])
    idx = int(binary_string, 2)
    overlap_H = fermionic_Hamiltonian[idx0][idx]
    H12 += y * overlap_H
    if j == 0:
        y0 = y
        
H21 = np.conjugate(H12)

S matrix!!

In [ ]:
S12 = y0
S21 = y0.conjugate()

### Calculating the Result

Finally, we are ready to solve the generalized eigenvalue problem using scipy

In [ ]:
from scipy import linalg

S = np.array([[S11, S12],[S21, S22]])
H = np.array([[H11, H12],[H21, H22]])

evals = linalg.eigvals(H, S)
energy_CBVQE = np.min(evals).real

## DISCUSSION

ADD PLOTS AND EXPLANATION